In [ ]:
T, image = cv2.threshold(gray, 100, 255,
	cv2.THRESH_BINARY_INV)
#cv2_imshow(threshInv)
image=255-image
image = cv2.rotate(image,cv2.ROTATE_90_CLOCKWISE)
#cv2_imshow(image)
#Line Segmentation
ret, thresh2 = cv2.threshold(image, 150, 255, cv2.THRESH_BINARY_INV)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (20,2))
#Have to change Kernel Value to obtain correct mask for different images
mask = cv2.morphologyEx(thresh2, cv2.MORPH_DILATE, kernel)
cv2_imshow(mask)
bboxes = []
bboxes_img = image.copy()
contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = contours[0] if len(contours) == 2 else contours[1]
for cntr in contours:
    x,y,w,h = cv2.boundingRect(cntr)
    cv2.rectangle(bboxes_img, (x, y), (x+w, y+h), (0,0,255), 1)
    bboxes.append((x,y,w,h))
for j in range(len(bboxes)):
    (x,y,w,h) = bboxes[j]
    crop = image[y:y+h, x:x+w]
    cv2_imshow(crop)
    #cv2.imwrite('/content/drive/MyDrive/LineSegmentation/B32{}.jpg'.format(j),crop)
    print('--------------------------------------')

In [ ]:
#Character Segmentation
from skimage.io import imread
from skimage.color import rgb2gray
import matplotlib.pyplot as plt

plt.imshow(crop, cmap="gray")
plt.show()
import numpy as np
from skimage.filters import threshold_otsu
from skimage.filters import gaussian

#binarize the image, guassian blur will remove any noise in the image
thresh = threshold_otsu(gaussian(crop))
binary = crop > thresh


# find the vertical projection by adding up the values of all pixels along rows
vertical_projection = np.sum(binary, axis=0)

# plot the vertical projects
fig, ax = plt.subplots(nrows=2)
plt.xlim(0, crop.shape[1])
ax[0].imshow(binary, cmap="gray")
ax[1].plot(vertical_projection)
#Continuation of Word Segmentation, works well if there is more than 1 word
height = crop.shape[0]
width=crop.shape[1]
#print(width)
#print(height)
## we will go through the vertical projections and 
## find the sequence of consecutive white spaces in the image
whitespace_lengths = []
whitespace = 0
print(vertical_projection)
for vp in vertical_projection:
    if vp == height:
        whitespace = whitespace + 1
    elif vp != height:
        if whitespace != 0:
            whitespace_lengths.append(whitespace)
        whitespace = 0 # reset whitepsace counter. 
if whitespace!=0:
  whitespace_lengths.append(whitespace)
print("whitespaces:", whitespace_lengths)
avg_white_space_length = min(whitespace_lengths)
print("average whitespace lenght:", avg_white_space_length)
whitespace_length = 0
divider_indexes = []
divider_indexes.append(0)
for index, vp in enumerate(vertical_projection):
    if vp == height:
        whitespace_length = whitespace_length + 1
    elif vp != height:
        if whitespace_length != 0 and whitespace_length >= avg_white_space_length:
            print(whitespace_length)
            divider_indexes.append(index-int(whitespace_length/2))
        whitespace_length = 0 # reset it
divider_indexes.append(width)            
#print(divider_indexes)
divider_indexes = np.array(divider_indexes)
dividers = np.column_stack((divider_indexes[:-1],divider_indexes[1:]))
#print(dividers)
fig, ax = plt.subplots(nrows=len(dividers), figsize=(5,10))
if len(dividers)==1:
  ax.imshow(crop[:,dividers[0][0]:dividers[0][1]], cmap="gray")
else:
    for index, window in enumerate(dividers):
      #ax[index].axis("off")
      ax[index].imshow(crop[:,window[0]:window[1]], cmap="gray")